In [0]:
dbutils.library.installPyPI("mlflow")
#dbutils.library.restartPython()
from pyspark.sql.types import StructType
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *      # for window() function
from typing import List
from pyspark.sql.types import *
import pandas as pd
import time
from datetime import datetime, timedelta, timezone
import mlflow
from pyspark.sql.types import StructType
import dateutil.parser
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.types import StructType

version = "v-17"

In [0]:
%sql
DROP TABLE raw_log_data_delta_PN_;
DROP TABLE anomalies_data_delta_PN_;

In [0]:
%sql
CREATE TABLE raw_log_data_delta_PN_ (
  account_id STRING,
  agent_id STRING,
  event STRING,
  timestamp TIMESTAMP
 )
USING DELTA;


CREATE TABLE anomalies_data_delta_PN_ (
  user_id STRING,
  Ips LONG,
  prediction DOUBLE
)
USING DELTA;


In [0]:
#57c7413abca837e974000009
inputPath = "dbfs:/mnt/kafka_raw/57c7413abca837e974000009/"


schema_raw_logs = (  StructType()
  .add("account_id","string")
  .add("agent_id","string")
  .add("event","string")
  .add("timestamp","timestamp") 
)


#reads From Kafka
eventsDF = (spark.readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "2.tcp.ngrok.io:16850")
  .option("subscribe", "event-splitter.audit_logs")
  #.schema(schema_raw_logs)
  #.json(inputPath)
  .load())
           
#writes to Raw table
(eventsDF.writeStream
  .outputMode("append")
  .option("checkpointLocation", "/mnt/delta/events/_checkpoints/etl-from-json_PN_"+version)
  .table("raw_log_data_delta_PN_")
)




Out[13]: <pyspark.sql.streaming.StreamingQuery at 0x7f35ac93b050>

In [0]:
#We may need to user OPTIMIZE, which deals with small files, merge them and compact them into larger files
raw_data = spark.readStream.format("delta").table("raw_log_data_delta_PN_")



In [0]:
%sql
select count(*) from raw_log_data_delta_PN_

count(1)
7497603


In [0]:
fullschema = (  StructType()
  .add("logger_event_id", "string")
  .add("logger_timestamp","timestamp")                
  .add("account_id","string")
  .add("agent_id","string")
  .add("event",StructType())
         .add("actor",StructType()
             .add("user_id","string")
             .add("ip_addresses",ArrayType(StringType()))
             .add("session_id","string")
             .add("impersonated_user_id","string")
             .add("id","string")
             .add("type","string")
             .add("user_agent","string")
         )
         .add("account_id","string")
         .add("event_type","string")
         .add("audit",StructType()
              .add("severity","string")
              .add("resource_id","string")
              .add("operation","string")
              .add("timestamp","timestamp")
              .add("status","string")
         )
         .add("logger_event_id","string")     
         .add("object",StructType())
         .add("timestamp","timestamp") 
   
   .add("timestamp","timestamp") 
)


In [0]:

run_id = "09840597c6e04f279aaa27be313c6e73"
model_uri = "runs:/" + run_id + "/sklearn-model"
model = mlflow.pyfunc.spark_udf(spark, model_uri)


filtered_data = (raw_data
                  .select( "timestamp"    ,from_json("event", fullschema).alias("data"))
                  .withColumn("timestamp",to_timestamp(to_date("timestamp","yyyy-MM-DD"),"yyyy-MM-DD"))
                  .select( "timestamp" ,   "data.actor.user_id",  "data.actor.ip_addresses")       
                  .where(col("user_id").isNotNull())
                  #.where(col("timestamp") >= datetime.now().astimezone(timezone.utc).strftime("%Y-%m-%dT00:00:00.000+0000") )
                  .withWatermark("timestamp", "24 hours")
                  .groupBy(col("timestamp"),"user_id").agg(approx_count_distinct('ip_addresses').alias('Ips'))
                  .select("user_id","Ips")
                  .withColumn("prediction", model("Ips"))
                )





In [0]:
display(filtered_data)

user_id,Ips,prediction
5f205f914b17e30007dc3c81,1,1.0
5e13afb5c67cce000419d11a,1,1.0
5d64329d43b1ed4d122505a1,1,1.0
5e583722bfdf8400092c7d51,1,1.0
5ed1294f190982000bbc0ea9,1,1.0
5c508601b496c300052c14c4,1,1.0
5ef4c08950879911e87c82ca,1,1.0
5c58fd98f86e7b00057ce703,1,1.0
5e39f2ea4aa42c0bfb88cab3,1,1.0
5e847e47bf34920009f53c49,2,1.0


In [0]:
#write data to anomalies' table
(filtered_data.writeStream
  .outputMode("append")
  .option("checkpointLocation", "/mnt/delta/events/_checkpoints/anomalies_"+version)
  .table("anomalies_data_delta_PN_")
)


Out[9]: <pyspark.sql.streaming.StreamingQuery at 0x7f82b9f56250>

In [0]:
#We may need to user OPTIMIZE, which deals with small files, merge them and compact them into larger files
anomalies = spark.readStream.format("delta").table("anomalies_data_delta_PN_")


filtered_anomalies = (anomalies
                        .select("user_id","Ips","prediction")    
                      )
display(filtered_anomalies)

In [0]:
%sql
select * from anomalies_data_delta_PN_

user_id,Ips,prediction
5f1f1d8d6832770c27107572,1,1.0
5ed1506507c0a43c569b3535,1,1.0
5c7f2f90b30e9d000803f268,1,1.0
5d8a3aa345ac1d000d61c3d9,1,1.0
5ed12543e631d2000a3c3cdd,1,1.0
5c58ee1556796f000528137f,1,1.0
5ef0e732c4b692000c7e864e,1,1.0
5ef34d4d63c1a6628ef6e07b,1,1.0
5ec2e1eacef5c3000abe4e0f,2,1.0
5e14011027f870000a50225b,2,1.0


In [0]:
%sql
select min(timestamp), max(timestamp) from raw_log_data_delta_PN_ where timestamp < to_date("2020-10-08T10:00:00.000+0000")

min(timestamp),max(timestamp)
2020-08-17T20:34:00.000+0000,2020-08-31T23:59:58.930+0000
